In [1]:
import librosa
from IPython.display import Audio
import pathlib
from pathlib import Path
from fastai.vision.all import *
from fastai.vision.widgets import *

C:\Users\Jens\anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Jens\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# Supress warnings that appear when uploading and classifying audio.
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# !sudo apt install ffmpeg
# !pip install torchaudio ipywebrtc
# !jupyter nbextension enable --py widgetsnbextension

In [4]:
from ipywebrtc import AudioRecorder, CameraStream

In [5]:
# The paths to where the recordings are being saved
data_path = Path('../../data/')
data_path.mkdir(exist_ok=True)
audio_path = data_path/'audio'
audio_path.mkdir(exist_ok=True)

In [6]:
# Creating the recorder to put in a widget later
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)

In [7]:
path = Path('../')
model_path = path/'models'

try:
    learn_inf = load_learner(model_path/'model_V2.pkl')
except:
    posix_backup = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    learn_inf = load_learner(model_path/'model_V2.pkl')
    pathlib.PosixPath = posix_backup
    

In [8]:
btn_upload = widgets.FileUpload(accept=".wav,.mp3")
out_rec = widgets.Output()
out_pl = widgets.Output()
lbl_pred = widgets.Label()
lbl_pred.value = 'Please select audio'
btn_run = widgets.Button(description='Classify recorded')

In [9]:
out_rec.clear_output()
with out_rec: display(recorder)

In [10]:
def log_mel_spec_tfm(audio, dst_path = path/'../data/imgs/uploaded'):
    data, sample_rate = audio
    
    n_fft = 1024
    hop_length = 512
    n_mels = 80
    fmin = 20
    fmax = sample_rate / 2 
    
    mel_spec_power = librosa.feature.melspectrogram(data, sr=sample_rate, n_fft=n_fft, 
                                                    hop_length=hop_length, 
                                                    n_mels=n_mels, power=2.0, 
                                                    fmin=fmin, fmax=fmax)
    
    #mel_spec_power = librosa.feature.melspectrogram(x, sr=sample_rate)
    
    mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max)
    
    
    dst_path.mkdir(exist_ok=True)
    try:
        fname = list(btn_upload.value)[0]
    except:
        fname = 'file.wav'
    
    plt.imsave(dst_path / (fname[:-4] + '.png'), mel_spec_db)
    
    return dst_path / (fname[:-4] + '.png')

In [11]:
def on_click(change):
    out_pl.clear_output()
    with out_pl: display(Audio(btn_upload.data[-1]))
    dst_path = path/'../data/imgs/uploaded'
    audio = librosa.load(io.BytesIO(btn_upload.data[-1]))
    audio_img = log_mel_spec_tfm(audio, dst_path)
    pred,pred_idx,probs = learn_inf.predict(audio_img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [12]:
def on_click_live(change):
    out_pl.clear_output()
    update_live_audio()
    audio = librosa.load(audio_path/'file.wav')
    x, sr = audio
    with out_pl: display(Audio(data=x, rate=sr))
    dst_path = path/'../data/imgs/uploaded'
    audio_img = log_mel_spec_tfm(audio, dst_path)
    pred,pred_idx,probs = learn_inf.predict(audio_img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [13]:
# Saves the audio file from the recorder as wav and puts it in a player
def update_live_audio():
    with open(audio_path/'recording.webm', 'wb') as f:
        f.write(recorder.audio.value)
    !ffmpeg -i ../../data/audio/recording.webm -ac 1 -f wav ../../data/audio/file.wav -y -hide_banner -loglevel panic
    sig, sr = librosa.load(audio_path/'file.wav')
    print(sig.shape)
    Audio(data=sig, rate=sr)

In [14]:
btn_upload.observe(on_click, names=['data'])

In [15]:
btn_run.on_click(on_click_live)

In [16]:
widgets.VBox([widgets.Label('Select your audio by using "upload" or record by tapping the record button followed by "Classify recording"'),
      out_rec, btn_run, btn_upload, out_pl, lbl_pred])

In [17]:
#!pip install voila
#!jupyter serverextension enable voila --sys-prefix